### Imports and constants

In [11]:
# imports
import pandas as pd
from general_data_editing import process_excel_multiple_sheets_to_csv
import os

In [96]:
input_file_responses = "data/FIN-RawData-FindingVulnerabilities-Global_manual-editedV2.xlsx"  # input file
input_file_ground_truth = "data/FIN-FindingVulnerabilities-GroundTruth_good_to_use.xlsx"  # input file ground truth
input_file_ground_truth_code_lines = "data/ground_truth_code_lines.xlsx"
responses_step1_file = "data/responses_step1.csv"  # .xlsx to csv of respondents
responses_step2_file = "data/responses_step2.csv"  # Generating first properties

### Loading Excel- and csv files

In [41]:
process_excel_multiple_sheets_to_csv(input_file_responses, responses_step1_file)

In [42]:
df_ground_truth = pd.read_excel(input_file_ground_truth)
df_gt_code_lines = pd.read_excel(input_file_ground_truth_code_lines)
df_responses = pd.read_csv(responses_step1_file)

In [43]:
print(df_ground_truth)
print(df_gt_code_lines)

  Groupnr                                               Path  \
0  Group1  [368,383,384,385,386,388, 432, 846, 853, 854, ...   
1  Group2  [76, 78, 79, 80, 81, 83, 95, 108, 113, 114, 11...   
2  Group3  [76, 78, 79, 80, 81, 83, 95, 108, 113, 114, 11...   
3  Group4  [368,383,384,385,386,388, 432, 846, 853, 854, ...   
4  Group5  [368,383,384,385,386,388, 432, 846, 853, 854, ...   
5  Group6  [76, 78, 79, 80, 81, 83, 95, 108, 113, 114, 11...   

                        User                   XSS              DoS  
0       [59, 61, 62, 87, 90]  [441, 444, 450, 451]       [48,50,51]  
1  [144, 147, 151, 287, 290]      [73, 74, 75, 76]  [459, 462, 463]  
2       [59, 61, 62, 87, 90]  [441, 444, 450, 451]  [459, 462, 463]  
3  [144, 147, 151, 287, 290]      [73, 74, 75, 76]       [48,50,51]  
4       [59, 61, 62, 87, 90]      [73, 74, 75, 76]  [459, 462, 463]  
5  [144, 147, 151, 287, 290]  [441, 444, 450, 451]       [48,50,51]  
   type                                              slice  \

In [75]:
difficulties_list = ["very easy", "easy", "neither hard nor easy", "hard", "very hard"]
confidence_list = ["0-20%", "20-40%", "40-60%", "60-80%", "80%-100%"]

## Pipeline itself

New csv file for responses, more detailed.

In [102]:
columns_for_new_csv = [
        'respondent_id',  # Data direct from raw data
        'duration_of_experiment',
        'which_group',
        'exp_coding',
        'exp_working_java',
        'exp_finding_vuln',
        'path_lines',
        'path_lines_correctness',
        'path_lines_type',
        'path_motivation',
        'path_difficulty',
        'path_confidence',
        'userinjection_lines',
        'userinjection_lines_correctness',
        'userinjection_lines_type',
        'userinjection_motivation',
        'userinjection_difficulty',
        'userinjection_confidence',
        'xss_lines',
        'xss_lines_correctness',
        'xss_lines_type',
        'xss_motivation',
        'xss_difficulty',
        'xss_confidence',
        'dos_lines',
        'dos_lines_correctness',
        'dos_lines_type',
        'dos_motivation',
        'dos_difficulty',
        'dos_confidence',
        'familiar_java',
        'familiar_vuln',
        'proc_understand',
        'proc_time',
        'proc_training',
        'avg_correctness_slice',
        'avg_correctness_full'
    ]

In [103]:
new_df = pd.DataFrame(columns=columns_for_new_csv)

In [104]:
def calc_lines_correctness(actual_lines: str, lines_raw: str) -> float:
    try:
        actual_lines_list = list(map(int, actual_lines.strip("[").strip("]").replace(",", " ").split()))

        lines_raw_list = list(map(int, filter(None, map(str.strip, lines_raw.replace(",", " ").replace("-"," ").replace(":", " ").replace(";", " ").split()))))

        # Calculate overlap between the two lists
        correct_lines = set(actual_lines_list).intersection(set(lines_raw_list))

        # Calculate the percentage of matching lines relative to the length of actual_lines_list
        correctness = len(correct_lines) / len(actual_lines_list) if actual_lines_list else 0.0

        return correctness
    except (ValueError, ZeroDivisionError) as e:
        # Handle potential conversion errors or division by zero
        print(f"Error processing input: {e}")
        return 0.0


In [105]:
def split_and_convert(value):
    return list(map(int, value.strip("[").strip("]").replace(",", " ").replace("-"," ").replace(";", " ").replace(":", " ").split()))

def extract_code_file_type(df, type_value, actual_lines):
    # Filter row based on 'type'
    _row = df[df['type'] == type_value]

    if row.empty:
        return "Type not found"

    try:
        # Convert the actual_lines string to a list of integers
        actual_lines_list = split_and_convert(actual_lines)

        # Convert 'slice' and 'full' columns to lists of integers
        slice_list = split_and_convert(_row.iloc[0]['slice'])
        full_list = split_and_convert(_row.iloc[0]['full'])

        # Check for exact matches
        slice_match = set(slice_list) == set(actual_lines_list)
        full_match = set(full_list) == set(actual_lines_list)

        # Determine which column matches exactly
        if slice_match:
            return 'slice'
        elif full_match:
            return 'full'
        else:
            return 'NaN'
    except ValueError as e:
        print(f"Error processing input: {e}")
        return 'NaN'

## Step 2 code

In [106]:
for index, row in df_responses.iterrows():
    respondent_id = row['ResponseId']
    duration_of_experiment = row['Duration (in seconds)']
    which_group = row['Which Group']
    exp_coding = row['Coder Experience']
    exp_working_java = row['WorkingJava']
    exp_finding_vuln = row['FindingVulns']

    total_correctness_slice = 0.0
    slice_count = 0
    total_correctness_full = 0.0
    full_count = 0

    actual_lines_row = df_ground_truth[df_ground_truth['Groupnr'] == which_group]

    # path
    path_lines = str(row['PathLines'])
    path_actual_lines = str(actual_lines_row.iloc[0]['Path']) # don't put in columns
    path_lines_correctness = calc_lines_correctness(path_actual_lines, path_lines)
    path_lines_type = extract_code_file_type(df_gt_code_lines, 'path', path_actual_lines)
    if path_lines_type == 'slice':
        slice_count += 1
        total_correctness_slice += path_lines_correctness
    elif path_lines_type == 'full':
        full_count += 1
        total_correctness_full += path_lines_correctness
    path_motivation = row['PathMotivation']
    path_difficulty = difficulties_list.index(row["Difficulty.Marks_1"].lower()) + 1
    path_confidence = confidence_list.index(row["Difficulty.Correct_1"].lower()) + 1

    # userinjection
    userinjection_lines = str(row['InjectionLines'])
    userinjection_actual_lines = str(actual_lines_row.iloc[0]['User']) # don't put in columns
    userinjection_lines_correctness = calc_lines_correctness(userinjection_actual_lines, userinjection_lines)
    userinjection_lines_type = extract_code_file_type(df_gt_code_lines, 'user', userinjection_actual_lines)
    if userinjection_lines_type == 'slice':
        slice_count += 1
        total_correctness_slice += userinjection_lines_correctness
    elif userinjection_lines_type == 'full':
        full_count += 1
        total_correctness_full += userinjection_lines_correctness
    userinjection_motivation = row['InjectionMotivation']
    userinjection_difficulty = difficulties_list.index(row["Difficulty.Marks_2"].lower()) + 1
    userinjection_confidence = confidence_list.index(row["Difficulty.Correct_2"].lower()) + 1

    # xss
    xss_lines = str(row['XSSLines'])
    xss_actual_lines = str(actual_lines_row.iloc[0]['XSS']) # don't put in columns
    xss_lines_correctness = calc_lines_correctness(xss_actual_lines, xss_lines)
    xss_lines_type = extract_code_file_type(df_gt_code_lines, 'xss', xss_actual_lines)
    if xss_lines_type == 'slice':
        slice_count += 1
        total_correctness_slice += xss_lines_correctness
    elif xss_lines_type == 'full':
        full_count += 1
        total_correctness_full += xss_lines_correctness
    xss_motivation = row['XSSMotivation']
    xss_difficulty = difficulties_list.index(row["Difficulty.Marks_3"].lower()) + 1
    xss_confidence = confidence_list.index(row["Difficulty.Correct_3"].lower()) + 1

    # dos
    dos_lines = str(row['DoSLines'])
    dos_actual_lines = str(actual_lines_row.iloc[0]['DoS']) # don't put in columns
    dos_lines_correctness = calc_lines_correctness(dos_actual_lines, dos_lines)
    dos_lines_type = extract_code_file_type(df_gt_code_lines, 'dos', dos_actual_lines)
    if dos_lines_type == 'slice':
        slice_count += 1
        total_correctness_slice += dos_lines_correctness
    elif dos_lines_type == 'full':
        full_count += 1
        total_correctness_full += dos_lines_correctness
    dos_motivation = row['DoSMotivation']
    dos_difficulty = difficulties_list.index(row["Difficulty.Marks_4"].lower()) + 1
    dos_confidence = confidence_list.index(row["Difficulty.Correct_4"].lower()) + 1


    familiar_java = row["Familiar.Java"]
    familiar_vuln = row["Familiar.Vuln"]
    proc_understand = row["Process.Understand"]
    proc_time = row["Process.Time"]
    proc_training = row["Process.Training"]

    avg_correctness_slice = None
    avg_correctness_full = None


    if slice_count > 0:
        avg_correctness_slice = total_correctness_slice / slice_count

    if full_count > 0:
        avg_correctness_full = total_correctness_full / full_count

    data_to_add = [
        respondent_id, duration_of_experiment, which_group, exp_coding, exp_working_java, exp_finding_vuln,
        path_lines, path_lines_correctness, path_lines_type, path_motivation, path_difficulty, path_confidence,
        userinjection_lines, userinjection_lines_correctness, userinjection_lines_type, userinjection_motivation, userinjection_difficulty, userinjection_confidence,
        xss_lines, xss_lines_correctness, xss_lines_type, xss_motivation, xss_difficulty, xss_confidence,
        dos_lines, dos_lines_correctness, dos_lines_type, dos_motivation, dos_difficulty, dos_confidence,
        familiar_java, familiar_vuln, proc_understand, proc_time, proc_training,
        avg_correctness_slice, avg_correctness_full
    ]
    new_df.loc[len(new_df)] = data_to_add


new_df.to_csv(responses_step2_file, index=False)


Error processing input: invalid literal for int() with base 10: 'The'
